## MCP Servers with Foundry Agents

![image](./Assets/image.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
mcp_server_name = os.getenv("MCP_SERVER_NAME")

### Setting up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Creating the MCP Server Connection ID

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == mcp_server_name:
        connection_id = connection.id
        break

print(f"The MCP Server Connection ID is: {connection_id}")

### Creating the MCP Tool Spec

In [ ]:
tool = MCPTool(
    server_label = "microsoft_learn_server",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="never",
    project_connection_id=connection_id
)

### Creating the MCP Agent

In [ ]:
agent = project_client.agents.create_version(
    agent_name="MCP-Agent",
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are an intelligent assistant that can interact with the Microsoft Learn MCP server to provide users with relevant learning resources and information about Microsoft technologies.",
        tools=[tool],
    )
)

print(f"Created MCP Agent with ID: {agent.id}")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat with the Agent

In [ ]:
user_query = "Can you pls help me with the latest ai foundry SDK code samples?" 

# Can also try this query: "Find me learning paths on Azure AI services for building intelligent applications."

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": "MCP-Agent",
            "type": "agent_reference"
        }
    },
    input=user_query
)

print(f"Agent Response: {response.output_text}")